In [1]:
%matplotlib inline
import numpy as np
import seaborn as sns
from ipywidgets import interactive
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation

import random
from itertools import count
import pandas as pd
from scipy.stats import norm
from random import sample
from scipy.spatial import distance

from matplotlib import animation



In [2]:
nn=91  #Change it according to the name of the instance that you want to analyze.

file_posterior='input'+ '_'+str(int(nn))+'.csv'
data_historical=pd.read_csv(file_posterior, header=None)  
set_observations=data_historical.to_numpy()


L=2000
H=2000

prior_theta1=L*(0.5)
prior_theta2=H*(0.5)
sd_prior=800



##### poserior likelihood evaluation function

def f(time_step):
    
    set_observations_t=set_observations[0:time_step,:]

       
    def posterior_evaluation(theta1,theta2,set_observations_t):
        
        
        ones_array=set_observations_t[set_observations_t[:,2]==1]
        zeros_array=set_observations_t[set_observations_t[:,2]==0]
    
        if ((set_observations_t.shape[0] -ones_array.shape[0])>75):
           
            
           hj=zeros_array.shape[0]
           
           new_zeros_array=zeros_array[sample(list(range(hj)),75),:]
            
           sub_set_observations=np.concatenate((new_zeros_array, ones_array), axis=0)   
          
            
            
        else:
            
            sub_set_observations=set_observations_t
    
        
        # =Theta_vector[0]
        # theta2=Theta_vector[1]
        
        f_theta1=norm.pdf(theta1, loc=prior_theta1, scale=sd_prior)
        f_theta2=norm.pdf(theta1, loc=prior_theta2, scale=sd_prior)
        
        temp=1
        
        for i in range(int(sub_set_observations.shape[0])):
            
            if sub_set_observations[i,2]==1:
                
                temp*= np.exp(-(distance.euclidean((sub_set_observations[i,0],sub_set_observations[i,1] ), (theta1,theta2)))*(1/100)) 
                
            else:
        
                temp*= 1-np.exp(-(distance.euclidean((sub_set_observations[i,0],sub_set_observations[i,1] ), (theta1,theta2)))*(1/100))
        
        
        
        
        return temp*f_theta1*f_theta2


    
    x = np.array(range(0,2000,40))
    y = np.array(range(0,2000,40))
    result = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
    
    montecarlo=np.zeros((result.shape[0],3))
    montecarlo[:,0]=result[:,0]
    montecarlo[:,1]=result[:,1]
    
    
    for k in range(result.shape[0]):
        
        montecarlo[k,2]=posterior_evaluation(montecarlo[k,0],montecarlo[k,1],set_observations_t)
        

    df_mc=pd.DataFrame(montecarlo, columns=['x','y','z'])



    pivoted = df_mc.pivot(index="x", columns="y", values="z")
    
    return pivoted



def plotting(time_step=1):
    
    z=f(time_step)
    ax=sns.heatmap(z)
    
    
interactive_plot=interactive(plotting,time_step=(0,80,5) )

interactive_plot



interactive(children=(IntSlider(value=1, description='time_step', max=80, step=5), Output()), _dom_classes=('w…